In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import current_date, lit

In [0]:
df_reservas = (
    spark.read.table("production.raw.tb_reservas")
    .select(
        F.col("reserva_id"),
        F.col("hospede_id"),
        F.col("quarto_id"),
        F.col("hotel_id"),
        F.col("data_reserva"),
        F.col("data_checkin"),
        F.col("data_checkout"),
        F.col("canal_reserva"),
        F.col("status"),
        F.col("valor_total"),
        F.col("desconto"),
        F.col("observacoes")
    )
    .withColumn("data_reserva", F.to_date(F.col("data_reserva")))
    .withColumn("data_checkin", F.to_date(F.col("data_checkin")))
    .withColumn("data_checkout", F.to_date(F.col("data_checkout")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_reservas.limit(5))

In [0]:
catalog_table = "production.trusted.tb_reservas"

if spark.catalog.tableExists(catalog_table):
    delta_table = DeltaTable.forName(spark, catalog_table)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_reservas.alias("source"),
        "target.reserva_id = source.reserva_id"
    ).whenMatchedUpdate(
        set = {
            **{col: f"source.{col}" for col in df_reservas.columns if col != "update_date"},
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    (
        df_reservas
        .withColumn("update_date", lit(""))
        .write
        .format("delta")
        .saveAsTable(catalog_table)
    )